# RPC Bench Results Overview

`results/rpc_bench/**/rpc_bench_*.json` から各サーバーの RPC ベンチマーク結果を収集し、指標を可視化します。

In [10]:
from pathlib import Path
import json
import re
import numpy as np
import pandas as pd
import polars as pl
import altair as alt
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

alt.data_transformers.disable_max_rows()
pl.Config.set_tbl_rows(100)

DATA_DIR = (Path.cwd() / "processed" / "rpc_bench").resolve()
DATA_DIR.mkdir(parents=True, exist_ok=True)

In [11]:
IOR_JSON_PATTERN = re.compile(r"rpc_bench_(\d+)\.json$")


def resolve_results_root() -> Path:
    candidates = [
        Path("results"),
        Path("..") / "results",
        Path("../..") / "results",
    ]
    for candidate in candidates:
        if candidate.exists():
            return candidate.resolve()
    raise FileNotFoundError(
        "results ディレクトリが見つかりません。ノートブックの位置を確認してください。"
    )


results_root = resolve_results_root()
print(f"Using results directory: {results_root}")


def load_rpc_bench(root: Path) -> pl.DataFrame:
    records = []
    for path in sorted(root.glob("rpc_bench/**/results/rpc_bench_*.json")):
        try:
            data = json.loads(path.read_text())
        except (OSError, json.JSONDecodeError) as exc:
            print(f"⚠️ Failed to parse {path}: {exc}")
            continue
        rel_parts = path.relative_to(root).parts
        match = IOR_JSON_PATTERN.search(path.name)
        run_index = int(match.group(1)) if match else None
        base = {
            "results_file": str(path.relative_to(root)),
            "collection": "/".join(rel_parts[:-1]),
            "category": rel_parts[0] if len(rel_parts) > 0 else "",
            "experiment": rel_parts[1] if len(rel_parts) > 1 else "",
            "run": rel_parts[2] if len(rel_parts) > 2 else "",
            "timestamp": data.get("timestamp"),
            "total_servers": data.get("total_servers"),
            "ping_iterations": data.get("ping_iterations"),
            "run_index": run_index,
        }
        for server in data.get("server_results", []) or []:
            latency = server.get("latency", {}) or {}
            record = {
                **base,
                "server_node": server.get("server_node"),
                "operations": server.get("operations"),
                "duration_secs": server.get("duration_secs"),
                "iops": server.get("iops"),
                "miops": server.get("miops"),
                "latency_min_us": latency.get("min_us"),
                "latency_avg_us": latency.get("avg_us"),
                "latency_p50_us": latency.get("p50_us"),
                "latency_p99_us": latency.get("p99_us"),
                "latency_max_us": latency.get("max_us"),
            }
            records.append(record)
    if not records:
        return pl.DataFrame([])
    df = pl.DataFrame(records)
    df = df.sort(["experiment", "run", "server_node", "run_index"], maintain_order=True)
    return df


Using results directory: /work/0/NBB/rmaeda/workspace/rust/benchfs/results


In [12]:
rpc_df = load_rpc_bench(results_root)
rpc_summary_df = pl.DataFrame([])

if rpc_df.is_empty():
    print("No rpc_bench JSON files found under results.")
else:
    display(rpc_df.head().to_pandas())

,results_file,collection,category,experiment,run,timestamp,total_servers,ping_iterations,run_index,server_node,operations,duration_secs,iops,miops,latency_min_us,latency_avg_us,latency_p50_us,latency_p99_us,latency_max_us
0,rpc_bench/2025.11.05-18.23.45-default/2025.11....,rpc_bench/2025.11.05-18.23.45-default/2025.11....,rpc_bench,2025.11.05-18.23.45-default,2025.11.05-18.23.57-294601.nqsv-4,2025-11-05T09:39:14.732560316+00:00,191,1000000,1,node_1,1000000,3.513716,284598.978259,0.284599,2.0,3.0,3.0,3.0,2931.0
1,rpc_bench/2025.11.05-18.23.45-default/2025.11....,rpc_bench/2025.11.05-18.23.45-default/2025.11....,rpc_bench,2025.11.05-18.23.45-default,2025.11.05-18.23.57-294601.nqsv-4,2025-11-05T09:39:14.732560316+00:00,191,1000000,1,node_10,1000000,3.572751,279896.361856,0.279896,3.0,3.0,3.0,3.0,381.0
2,rpc_bench/2025.11.05-18.23.45-default/2025.11....,rpc_bench/2025.11.05-18.23.45-default/2025.11....,rpc_bench,2025.11.05-18.23.45-default,2025.11.05-18.23.57-294601.nqsv-4,2025-11-05T09:39:14.732560316+00:00,191,1000000,1,node_100,1000000,4.979831,200810.010109,0.200810,4.0,4.0,4.0,5.0,910.0
3,rpc_bench/2025.11.05-18.23.45-default/2025.11....,rpc_bench/2025.11.05-18.23.45-default/2025.11....,rpc_bench,2025.11.05-18.23.45-default,2025.11.05-18.23.57-294601.nqsv-4,2025-11-05T09:39:14.732560316+00:00,191,1000000,1,node_101,1000000,5.007937,199683.023687,0.199683,4.0,4.0,4.0,5.0,913.0
4,rpc_bench/2025.11.05-18.23.45-default/2025.11....,rpc_bench/2025.11.05-18.23.45-default/2025.11....,rpc_bench,2025.11.05-18.23.45-default,2025.11.05-18.23.57-294601.nqsv-4,2025-11-05T09:39:14.732560316+00:00,191,1000000,1,node_102,1000000,4.983325,200669.224922,0.200669,4.0,4.0,4.0,5.0,914.0


## Summary Metrics

実験×実行ごとの集計値を確認します。

In [13]:
if rpc_df.is_empty():
    print("No summary available.")
else:
    rpc_summary_df = (
        rpc_df
        .group_by(["experiment", "run", "timestamp"], maintain_order=True)
        .agg(
            pl.len().alias("servers"),
            pl.col("iops").mean().alias("iops_mean"),
            pl.col("iops").sum().alias("iops_total"),
            pl.col("iops").max().alias("iops_max"),
            pl.col("iops").min().alias("iops_min"),
            pl.col("latency_avg_us").mean().alias("latency_avg_us_mean"),
            pl.col("latency_p99_us").max().alias("latency_p99_us_max"),
            pl.col("latency_max_us").max().alias("latency_max_us"),
            pl.col("duration_secs").mean().alias("duration_secs_mean"),
        )
        .sort(["experiment", "run"])
    )
    display(rpc_summary_df.to_pandas())

,experiment,run,timestamp,servers,iops_mean,iops_total,iops_max,iops_min,latency_avg_us_mean,latency_p99_us_max,latency_max_us,duration_secs_mean
0,2025.11.05-18.23.45-default,2025.11.05-18.23.57-294601.nqsv-4,2025-11-05T09:39:14.732560316+00:00,191,215499.196581,4.116035e+07,285086.247390,185690.901253,3.963351,5.0,28009.0,4.718876
1,2025.11.05-18.23.45-default,2025.11.05-18.23.58-294602.nqsv-8,2025-11-05T09:55:23.337576280+00:00,383,208576.124429,7.988466e+07,286471.571513,179277.458900,4.041775,5.0,24009.0,4.849985
2,2025.11.05-18.23.45-default,2025.11.05-18.23.58-294603.nqsv-16,2025-11-05T10:28:43.387428375+00:00,767,201470.782169,1.545281e+08,280694.105403,175359.273339,4.451108,5.0,28007.0,4.996984


## Server Throughput by Run

ランごとのサーバー IOPS をヒートマップ的に可視化します。

In [17]:
if rpc_df.is_empty():
    print("No data to plot.")
else:
    chart_df = (
        rpc_df.select([
            "experiment",
            "run",
            "server_node",
            "iops",
            "latency_p99_us",
            "latency_max_us",
        ])
        .to_pandas()
        .assign(
            server_node=lambda df: df["server_node"].astype(str),
            iops=lambda df: pd.to_numeric(df["iops"], errors="coerce"),
            latency_p99_us=lambda df: pd.to_numeric(df["latency_p99_us"], errors="coerce"),
            latency_max_us=lambda df: pd.to_numeric(df["latency_max_us"], errors="coerce"),
        )
    )
    chart_df["miops"] = chart_df["iops"].fillna(0.0) / 1_000_000

    run_options = sorted(chart_df["run"].dropna().unique())
    if not run_options:
        print("No run identifiers available.")
    else:
        totals_df = (
            chart_df.groupby(["experiment", "run"], as_index=False)["miops"].sum()
            .rename(columns={"miops": "miops_total"})
        )
        totals_df["experiment_run"] = (
            totals_df["experiment"].astype(str)
            + " / "
            + totals_df["run"].astype(str)
        )
        run_node_counts = chart_df.groupby("run")["server_node"].nunique()

        def plot_run(run_value: str, top_n: int) -> None:
            subset = chart_df[chart_df["run"] == run_value].copy()
            available = int(run_node_counts.get(run_value, 0))
            if subset.empty:
                print(f"No data available for run: {run_value}")
                return

            top_n = max(1, min(top_n, available))
            subset = subset.sort_values("miops", ascending=False)

            latencies = subset["latency_p99_us"].fillna(0.0)
            latency_min = float(latencies.min()) if not latencies.empty else 0.0
            latency_max = float(latencies.max()) if not latencies.empty else latency_min + 1.0
            if np.isclose(latency_min, latency_max):
                latency_max = latency_min + 1.0

            top_subset = subset.head(top_n).copy()
            top_subset["is_other"] = False

            if available > top_n:
                others_miops = subset["miops"].iloc[top_n:].sum()
                others_latency = subset["latency_p99_us"].iloc[top_n:].mean()
                others_label = f"Other nodes ({available - top_n})"
                top_subset = pd.concat(
                    [
                        top_subset,
                        pd.DataFrame(
                            {
                                "server_node": [others_label],
                                "miops": [others_miops],
                                "latency_p99_us": [others_latency],
                                "latency_max_us": [np.nan],
                                "is_other": [True],
                            }
                        ),
                    ],
                    ignore_index=True,
                )

            norm = plt.Normalize(vmin=latency_min, vmax=latency_max)
            cmap = plt.cm.Reds
            colors = [
                "#888888"
                if flag
                else cmap(norm(lat if np.isfinite(lat) else latency_min))
                for flag, lat in zip(top_subset["is_other"], top_subset["latency_p99_us"])
            ]

            totals_sorted = totals_df.sort_values("miops_total", ascending=False)
            highlight_colors = [
                "#1f77b4" if run == run_value else "#d3d3d3"
                for run in totals_sorted["run"]
            ]

            fig_height = 2.5 + 0.35 * len(top_subset)
            fig, (ax_totals, ax_servers) = plt.subplots(
                2,
                1,
                figsize=(10, fig_height),
                gridspec_kw={"height_ratios": [1, 2]},
            )

            ax_totals.bar(
                totals_sorted["experiment_run"],
                totals_sorted["miops_total"],
                color=highlight_colors,
                edgecolor="black",
            )
            ax_totals.set_ylabel("Total MIOPS")
            ax_totals.set_title("Total Throughput by Experiment / Run")
            ax_totals.tick_params(axis="x", rotation=35)
            for label in ax_totals.get_xticklabels():
                label.set_ha("right")

            ax_servers.barh(
                top_subset["server_node"],
                top_subset["miops"],
                color=colors,
                edgecolor="black",
            )
            ax_servers.invert_yaxis()
            ax_servers.set_xlabel("Throughput (MIOPS)")
            ax_servers.set_ylabel("Server node")
            ax_servers.set_title(
                f"Server Throughput for Run: {run_value} (showing top {top_n} of {available})"
            )

            sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
            sm.set_array([])
            cbar = fig.colorbar(sm, ax=ax_servers, orientation="vertical")
            cbar.set_label("Latency p99 (us)")

            max_miops = float(top_subset["miops"].max()) if not top_subset.empty else 0.0
            offset = 0.01 if not np.isfinite(max_miops) or max_miops <= 0 else max_miops * 0.01
            for idx, (_, row) in enumerate(top_subset.iterrows()):
                ax_servers.text(
                    row["miops"] + offset,
                    idx,
                    f"{row['miops']:.3f} MIOPS",
                    va="center",
                    fontsize=9,
                )

            if available > top_n:
                others_total = subset["miops"].iloc[top_n:].sum()
                fig.text(
                    0.01,
                    0.02,
                    f"Aggregated {available - top_n} additional nodes → {others_total:.3f} MIOPS",
                    fontsize=9,
                )

            fig.tight_layout()
            display(fig)
            plt.close(fig)

        max_nodes = int(run_node_counts.max()) if not run_node_counts.empty else 1
        default_top = min(20, max_nodes)
        top_n_widget = widgets.IntSlider(
            value=default_top,
            min=1,
            max=max(1, max_nodes),
            step=1,
            description="Top N",
            continuous_update=False,
        )
        run_widget = widgets.Dropdown(options=run_options, description="Run")
        output = widgets.Output()

        def refresh_plot(*_):
            if run_widget.value is None:
                return
            available = int(run_node_counts.get(run_widget.value, 0))
            if available <= 0:
                output.clear_output(wait=True)
                with output:
                    print(f"No data available for run: {run_widget.value}")
                return
            if top_n_widget.value > available:
                top_n_widget.value = available
                return
            output.clear_output(wait=True)
            with output:
                plot_run(run_widget.value, top_n_widget.value)

        run_widget.observe(
            lambda change: refresh_plot() if change["name"] == "value" else None,
            names="value",
        )
        top_n_widget.observe(
            lambda change: refresh_plot() if change["name"] == "value" else None,
            names="value",
        )

        with output:
            plot_run(run_widget.value, top_n_widget.value)

        display(widgets.VBox([widgets.HBox([run_widget, top_n_widget]), output]))

## Latency Metrics

平均・p99・最大レイテンシのプロファイルを比較します。

In [15]:
if rpc_df.is_empty():
    print("No data to plot.")
else:
    latency_long = (
        rpc_df
        .select([
            "experiment",
            "run",
            "server_node",
            "latency_avg_us",
            "latency_p99_us",
            "latency_max_us",
        ])
        .melt(id_vars=["experiment", "run", "server_node"], value_vars=["latency_avg_us", "latency_p99_us", "latency_max_us"], variable_name="metric", value_name="latency_us")
    )
    latency_pdf = latency_long.to_pandas()
    latency_pdf["server_node"] = latency_pdf["server_node"].astype(str)
    run_options = sorted(latency_pdf["run"].unique())
    if not run_options:
        print("No runs available.")
    else:
        run_dropdown = alt.binding_select(options=run_options, name="Run")
        run_select = alt.selection_point(fields=["run"], bind=run_dropdown, value=[{"run": run_options[0]}])
        chart = (
            alt.Chart(latency_pdf)
            .add_params(run_select)
            .transform_filter(run_select)
            .mark_circle(size=70, opacity=0.85)
            .encode(
                x=alt.X("server_node:N", title="Server node"),
                y=alt.Y("latency_us:Q", title="Latency (us)", scale=alt.Scale(type="log")),
                color=alt.Color("metric:N", title="Metric"),
                tooltip=[
                    alt.Tooltip("experiment:N"),
                    alt.Tooltip("run:N"),
                    alt.Tooltip("server_node:N"),
                    alt.Tooltip("metric:N"),
                    alt.Tooltip("latency_us:Q", format=".1f"),
                ],
            )
            .properties(width=750, height=360)
        )
        display(chart)

/tmp/ipykernel_1554208/955644274.py:14: DeprecationWarning: `DataFrame.melt` is deprecated; use `DataFrame.unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  .melt(id_vars=["experiment", "run", "server_node"], value_vars=["latency_avg_us", "latency_p99_us", "latency_max_us"], variable_name="metric", value_name="latency_us")


alt.Chart(...)

## Export Processed Data

可視化に使用したテーブルを JSON として保存します。

In [16]:
if rpc_df.is_empty():
    print("No data to export.")
else:
    DATA_DIR.mkdir(parents=True, exist_ok=True)
    server_path = DATA_DIR / "rpc_server_results.json"
    server_path.write_text(rpc_df.to_pandas().to_json(orient="records", indent=2))
    print(f"Wrote {server_path}")
    if rpc_summary_df.is_empty():
        print("No summary table to export.")
    else:
        summary_path = DATA_DIR / "rpc_summary.json"
        summary_path.write_text(rpc_summary_df.to_pandas().to_json(orient="records", indent=2))
        print(f"Wrote {summary_path}")

Wrote /work/0/NBB/rmaeda/workspace/rust/benchfs/plot/processed/rpc_bench/rpc_server_results.json
Wrote /work/0/NBB/rmaeda/workspace/rust/benchfs/plot/processed/rpc_bench/rpc_summary.json
